https://www.kaggle.com/code/manovirat/timeseries-using-prophet-hyperparameter-tuning

In [1]:
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.model_selection import ParameterGrid
from prophet.diagnostics import performance_metrics
import random
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

In [2]:
params_grid = {'seasonality_mode':('multiplicative','additive'),
               # set changepoint_prior_scale to 0.5 with steps of 0.005
               'changepoint_prior_scale':[x for x in np.arange(0.001, 0.5, 0.1)],
               'seasonality_prior_scale':[x for x in np.arange(0.01, 10, 2.5)],
              'holidays_prior_scale':[x for x in np.arange(0.01, 10, 2.5)]}

grid = ParameterGrid(params_grid)
cnt = 0
for p in grid:
    cnt = cnt+1

print('Total Possible Models',cnt)

Total Possible Models 160


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_15min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date'])


In [5]:
# Feature Engineering

# Add a column with the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# convert date column to datetime without timezone
data['date'] = pd.to_datetime(data['date'],format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 

# Rename column count to y
data = data.rename(columns={'count': 'y'})
data = data.rename(columns={'date': 'ds'})


train_data = data

In [6]:
cv_folds = TimeSeriesSplit(n_splits=5)

#convert data to only date not time
cv_data = train_data.copy()

#convert to timestamp with date format
#cv_data.ds = pd.to_datetime(cv_data.ds).dt.date

print(cv_data.ds)   

#Define cutoff dates for cross validation at end of train_data/cv_folds
cutoffs = []
# for train_index, test_index in cv_folds.split(train_data):
#     cutoffs.append(train_data.ds.iloc[test_index].max())
#     #convert to timestamp
#     cutoffs = pd.to_datetime(cutoffs)
    
# print(cutoffs)

for i, (train_index, test_index) in enumerate(cv_folds.split(cv_data)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    #get the date for the last train index
    cutoffs.append(cv_data.ds.iloc[train_index].max())
    #only date not time
    #cutoffs = pd.to_datetime(cutoffs).date

0        2022-04-01 00:00:00
1        2022-04-01 00:15:00
2        2022-04-01 00:30:00
3        2022-04-01 00:45:00
4        2022-04-01 01:00:00
                ...         
35035    2023-03-31 22:45:00
35036    2023-03-31 23:00:00
35037    2023-03-31 23:15:00
35038    2023-03-31 23:30:00
35039    2023-03-31 23:45:00
Name: ds, Length: 35040, dtype: object
Fold 0:
  Train: index=[   0    1    2 ... 5837 5838 5839]
  Test:  index=[ 5840  5841  5842 ... 11677 11678 11679]
Fold 1:
  Train: index=[    0     1     2 ... 11677 11678 11679]
  Test:  index=[11680 11681 11682 ... 17517 17518 17519]
Fold 2:
  Train: index=[    0     1     2 ... 17517 17518 17519]
  Test:  index=[17520 17521 17522 ... 23357 23358 23359]
Fold 3:
  Train: index=[    0     1     2 ... 23357 23358 23359]
  Test:  index=[23360 23361 23362 ... 29197 29198 29199]
Fold 4:
  Train: index=[    0     1     2 ... 29197 29198 29199]
  Test:  index=[29200 29201 29202 ... 35037 35038 35039]


In [7]:
MASE_list = []
best_mase = 100000000
best_params = None
for p in grid:
    train = pd.DataFrame(train_data)
    test = pd.DataFrame()
    print(p)
    random.seed(0)
    train_model =Prophet(changepoint_prior_scale = p['changepoint_prior_scale'],
                        holidays_prior_scale = p['holidays_prior_scale'],
                        seasonality_prior_scale = p['seasonality_prior_scale'],
                        seasonality_mode = p['seasonality_mode'],
                        weekly_seasonality=True,
                        daily_seasonality = True,
                        yearly_seasonality = False,
                        )
    
    # add regressors
    train_model.add_regressor('hour_of_day')
    train_model.add_regressor('day_of_week')
    train_model.add_regressor('day_of_month')
    train_model.add_regressor('month_of_year')
    train_model.add_regressor('year')
    train_model.add_regressor('weekend')
    train_model.add_regressor('monday')
    train_model.add_regressor('tuesday')
    train_model.add_regressor('wednesday')
    train_model.add_regressor('thursday')
    train_model.add_regressor('friday')
    train_model.add_regressor('saturday')
    train_model.add_regressor('sunday')
    train_model.add_regressor('time')


    train_model.fit(train_data)
    
    df_cv = cross_validation(train_model, period='60 days', horizon='30 days', parallel="processes")

     # X and y to pandas dataframe
    X_train = pd.DataFrame(df_cv)

    y_true = df_cv['y'].values
    #make y_true as dataframe
    y_true = pd.DataFrame(y_true)

    y_pred = df_cv['yhat'].values
    #make y_pred as dataframe
    y_pred = pd.DataFrame(y_pred)

    MASE_list.append(mean_absolute_scaled_error(y_true, y_pred))
    


    # print('RMSE:',np.sqrt(MSE))
    # RMSE_list.append(np.sqrt(MSE))

# Find the best parameters
tuning_results = pd.DataFrame(grid)
tuning_results['mase'] = MASE_list
print(tuning_results)

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 0.01}


10:16:06 - cmdstanpy - INFO - Chain [1] start processing
10:16:07 - cmdstanpy - INFO - Chain [1] done processing
10:16:09 - cmdstanpy - INFO - Chain [1] start processing
10:16:09 - cmdstanpy - INFO - Chain [1] start processing
10:16:10 - cmdstanpy - INFO - Chain [1] start processing
10:16:10 - cmdstanpy - INFO - Chain [1] done processing
10:16:10 - cmdstanpy - INFO - Chain [1] start processing
10:16:10 - cmdstanpy - INFO - Chain [1] done processing
10:16:11 - cmdstanpy - INFO - Chain [1] start processing
10:16:11 - cmdstanpy - INFO - Chain [1] done processing
10:16:11 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
10:16:12 - cmdstanpy - INFO - Chain [1] start processing
10:16:12 - cmdstanpy - INFO - Chain [1] done processing
10:16:12 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
10:16:12 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 2.51}


10:20:51 - cmdstanpy - INFO - Chain [1] start processing
10:20:52 - cmdstanpy - INFO - Chain [1] done processing
10:20:54 - cmdstanpy - INFO - Chain [1] start processing
10:20:54 - cmdstanpy - INFO - Chain [1] done processing
10:20:54 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] done processing
10:20:55 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:55 - cmdstanpy - INFO - Chain [1] start processing
10:20:56 - cmdstanpy - INFO - Chain [1] start processing
10:20:58 - cmdstanpy - INFO - Chain [1] done processing
10:20:58 - cmdstanpy - INFO - Chain [1] done processing
10:20:59 - cmdstanpy - INFO - Chain [1] done processing
10:20:59 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization 

{'changepoint_prior_scale': 0.001, 'holidays_prior_scale': 0.01, 'seasonality_mode': 'multiplicative', 'seasonality_prior_scale': 5.01}


...

{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 5.01}


11:58:49 - cmdstanpy - INFO - Chain [1] start processing
11:58:54 - cmdstanpy - INFO - Chain [1] done processing
11:58:56 - cmdstanpy - INFO - Chain [1] start processing
11:58:56 - cmdstanpy - INFO - Chain [1] done processing
11:58:56 - cmdstanpy - INFO - Chain [1] start processing
11:58:57 - cmdstanpy - INFO - Chain [1] start processing
11:58:57 - cmdstanpy - INFO - Chain [1] done processing
11:58:57 - cmdstanpy - INFO - Chain [1] start processing
11:58:57 - cmdstanpy - INFO - Chain [1] start processing
11:58:58 - cmdstanpy - INFO - Chain [1] done processing
11:58:59 - cmdstanpy - INFO - Chain [1] done processing
11:59:05 - cmdstanpy - INFO - Chain [1] done processing


{'changepoint_prior_scale': 0.401, 'holidays_prior_scale': 7.51, 'seasonality_mode': 'additive', 'seasonality_prior_scale': 7.51}


11:59:07 - cmdstanpy - INFO - Chain [1] start processing
11:59:16 - cmdstanpy - INFO - Chain [1] done processing
11:59:17 - cmdstanpy - INFO - Chain [1] start processing
11:59:18 - cmdstanpy - INFO - Chain [1] done processing
11:59:18 - cmdstanpy - INFO - Chain [1] start processing
11:59:18 - cmdstanpy - INFO - Chain [1] start processing
11:59:19 - cmdstanpy - INFO - Chain [1] start processing
11:59:19 - cmdstanpy - INFO - Chain [1] done processing
11:59:19 - cmdstanpy - INFO - Chain [1] done processing
11:59:19 - cmdstanpy - INFO - Chain [1] start processing
11:59:20 - cmdstanpy - INFO - Chain [1] done processing
11:59:26 - cmdstanpy - INFO - Chain [1] done processing


     changepoint_prior_scale  holidays_prior_scale seasonality_mode  \
0                      0.001                  0.01   multiplicative   
1                      0.001                  0.01   multiplicative   
2                      0.001                  0.01   multiplicative   
3                      0.001                  0.01   multiplicative   
4                      0.001                  0.01         additive   
..                       ...                   ...              ...   
155                    0.401                  7.51   multiplicative   
156                    0.401                  7.51         additive   
157                    0.401                  7.51         additive   
158                    0.401                  7.51         additive   
159                    0.401                  7.51         additive   

     seasonality_prior_scale      mase  
0                       0.01  1.074007  
1                       2.51  0.989979  
2                       

In [8]:
best_params = grid[np.argmin(MASE_list)]
print('Best Parameters',best_params)
print('Best MASE',np.min(MASE_list))
print(MASE_list)

Best Parameters {'seasonality_prior_scale': 0.01, 'seasonality_mode': 'additive', 'holidays_prior_scale': 7.51, 'changepoint_prior_scale': 0.101}
Best MASE 0.977624726184889
[1.0740066754627058, 0.9899789936943866, 1.2370928504523488, 1.0147333868778876, 1.1776957991714705, 2.1511798992545375, 1.2676861234464105, 1.7715985849824427, 7.196076703161533, 1.077170587873993, 0.9856706107629679, 2.7064335964594926, 7.146207724352997, 1.4117734830068038, 1.1702857430437457, 7.308440946096777, 0.9929722081316645, 3.507466971789767, 1.2256425639322277, 1.0373436755841154, 1.3223001655691249, 1.3713523430598176, 7.318638439593749, 2.524883431582752, 7.224808500434247, 1.035068297975622, 0.9986459155981842, 0.9880582381242115, 1.3962693861967364, 1.1771398083894453, 1.2951358980608272, 7.059696493330854, 1.0414276081018177, 0.9951544223253556, 1.0288643342945363, 1.0087448095775093, 0.9964037058711059, 0.9931016266889473, 0.996606061973236, 0.9879299394258109, 1.019393549799009, 1.018882198903737